In [ ]:
from tbh import runner_tools as rt 
from tbh import plotting as pl

from importlib import reload
reload(rt);
reload(pl);

In [ ]:
from tbh import model as tbm 
reload(tbm)

params, priors, tv_params = rt.get_parameters_and_priors()
model_config = rt.DEFAULT_MODEL_CONFIG
model = tbm.get_tb_model(model_config, tv_params)

In [ ]:
from tbh.paths import DATA_FOLDER
sc_params, _, sc_tv_params = rt.get_parameters_and_priors(DATA_FOLDER / "scenario_1.xlsx")
sc_params = params | sc_params
sc_tv_params = tv_params | sc_tv_params
model_1 = tbm.get_tb_model(model_config, sc_tv_params)

In [ ]:
ow_params = {
    "raw_transmission_rate": 10.,
}
model.run(params | ow_params)

In [ ]:
model_1.run(params | ow_params)

In [ ]:
df = model.get_derived_outputs_df()
df['tb_incidence_per100k'].plot()

In [ ]:
df1 = model_1.get_derived_outputs_df()

In [ ]:
df['tb_incidence_per100k'].loc[2018:2030].plot()
df1['tb_incidence_per100k'].loc[2018:2030].plot()


## Metropolis calibration

In [ ]:
from estival.model import BayesianCompartmentalModel
from estival import targets as est

In [ ]:
import pandas as pd
targets = [
    est.NormalTarget("notifications", pd.Series([100], [2020]),stdev=20.)
]
bcm = BayesianCompartmentalModel(model, params, priors, targets)

In [ ]:
import multiprocessing as mp
mp.set_start_method("spawn")  # previously "forkserver"

import pytensor
pytensor.config.cxx = '/usr/bin/clang++'

In [ ]:
idata = rt.run_metropolis_calibration(bcm, draws=100, tune=50, chains=2, cores=1)